In [3]:
!unzip "dataset.zip"

Archive:  dataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of dataset.zip or
        dataset.zip.zip, and cannot find dataset.zip.ZIP, period.


In [2]:
!git lfs install
!git clone https://huggingface.co/liuhaotian/llava-v1.5-7b

Git LFS initialized.
Cloning into 'llava-v1.5-7b'...
remote: Enumerating objects: 19, done.
remote: Total 19 (delta 0), reused 0 (delta 0), pack-reused 19
Unpacking objects: 100% (19/19), 4.70 KiB | 801.00 KiB/s, done.
Filtering content: 100% (4/4), 4.63 GiB | 7.91 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.


In [ ]:
#!/bin/bash

# Set the prompt and model versions directly in the command
!deepspeed /LLaVA/llava/train/train_mem.py \
    --deepspeed /LLaVA/scripts/zero2.json \
    --lora_enable True \
    --lora_r 128 \
    --lora_alpha 256 \
    --mm_projector_lr 2e-5 \
    --bits 4 \
    --model_name_or_path /LLaVA/llava/llava-v1.5-7b \
    --version llava_llama_2 \
    --data_path /dataset/train/dataset.json \
    --validation_data_path /dataset/validation/dataset.json \
    --image_folder /dataset/images/ \
    --vision_tower openai/clip-vit-large-patch14-336 \
    --mm_projector_type mlp2x_gelu \
    --mm_vision_select_layer -2 \
    --mm_use_im_start_end False \
    --mm_use_im_patch_token False \
    --image_aspect_ratio pad \
    --group_by_modality_length True \
    --bf16 True \
    --output_dir /LLaVA/llava/checkpoints/llama-2-7b-chat-task-qlora \
    --num_train_epochs 500 \
    --per_device_train_batch_size 32 \
    --per_device_eval_batch_size 32 \
    --gradient_accumulation_steps 1 \
    --evaluation_strategy “epoch” \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-4 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True \
    --report_to wandb


In [ ]:
# Inference

!python run_llava.py --model-path /LLaVA/llava/checkpoints/llava-2-7b-chat-task-qlora/best_llava_eval_model_llava_lora
--model-base /LLaVA/llava/llava-v1.5-7b
--image-file /dataset/images/0f47c0b5-2c77-45e6-87b0-89af46e99500.jpg
--query “why was this photo taken?”
